In [2]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
import time

from openpyxl import Workbook

In [4]:
original_list = ["caresense2_meter_fullset"]                          # 수집하려는 품목들을 리스트로 만든다.
product_code = {"caresense2_meter_fullset" : 9355226}

In [5]:
# 위 정보를 이용해서 각 상품별 url_list를 만들어준다.
url_list = [] 
for lis in original_list:
    key = product_code[lis] 
    raw_url = f"https://www.coupang.com/np/categories/{key}?listSize=120&brand=&offerCondition=&filterType=&isPriceRange=false&minPrice=&maxPrice=&page="
    url_list.append(raw_url)

In [ ]:
for name, main_url in zip(original_list, url_list) : 
    print("*" * 50 + " " + f"{name}시작하겠습니다." + " " + "*" * 50)
    
    for i in range(1, 6):
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)   
        temp_url = main_url + f"{i}"
        driver.get(temp_url)
        time.sleep(10)
        print("*" * 50 + " " + str(i) + "Page start!" + " " + "*" * 50)

        wb = Workbook()
        ws = wb.create_sheet(name)
        wb.remove_sheet(wb["Sheet"])
        ws.append(["상품명", "가격", "배송기한", "상세URL"]) 

        try :
            product = driver.find_element(By.ID, "productList")
            lis = product.find_elements(By.CLASS_NAME, "baby-product")

            for li in lis:
                try : 
                    product = li.find_element(By.CLASS_NAME, "name").text
                    price = li.find_element(By.CLASS_NAME, "price-value").text
                    delivery = li.find_element(By.CLASS_NAME, "delivery").text
                    product_url = li.find_element(By.CLASS_NAME, "baby-product-link").get_attribute("href")
                    ws.append([product, price, delivery, product_url])
                    print("product : ", product)
                    print("price : ", price)
                    print("delivery : ", delivery)
                    print("url : ", product_url)
                except Exception:
                    pass 
            print("*" * 50 + " " + str(i) + "Page end!" + " " + "*" * 50)
            time.sleep(5)
            wb.save(f"/Users/kds/Documents/coupang/{name}_{i}page.xlsx")
            wb.close()
            
            driver.quit()
            
        except NoSuchElementException :
            print("에러가 발생하여 정상적으로 데이터가 수집되지 않았습니다.")
            
    print(f"*" * 50 + "모든 데이터 수집을 마쳤습니다. 감사합니다. " + " " + "*" * 50)